In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings

# Part 2  The Fcators

## 1. Analyze the factors, similar to how you analyzed the three Fama-French factors in Homework 4. You now have three additional factors, so let's compare there univariate statistics.

In [51]:
df = pd.read_excel("factor_pricing_data.xlsx",sheet_name = 1)
df.set_index('Date',inplace = True)
df

,MKT,SMB,HML,RMW,CMA,UMD
Date,,,,,,
1980-01-31,0.0551,0.0183,0.0175,-0.0170,0.0164,0.0755
1980-02-29,-0.0122,-0.0157,0.0061,0.0004,0.0268,0.0788
1980-03-31,-0.1290,-0.0693,-0.0101,0.0146,-0.0119,-0.0955
1980-04-30,0.0397,0.0105,0.0106,-0.0210,0.0029,-0.0043
1980-05-31,0.0526,0.0211,0.0038,0.0034,-0.0031,-0.0112
...,...,...,...,...,...,...
2022-04-30,-0.0946,-0.0040,0.0619,0.0363,0.0592,0.0489
2022-05-31,-0.0034,-0.0006,0.0841,0.0144,0.0398,0.0248
2022-06-30,-0.0843,0.0130,-0.0597,0.0185,-0.0470,0.0079


In [50]:
#1.
ss = (df.mean()*12).to_frame(name = 'Mean')
ss['vol'] = (df.std()*np.sqrt(12))
ss['Sharpe'] = ss['Mean']/ss['vol']

ss

,Mean,vol,Sharpe
MKT,0.083123,0.156687,0.530507
SMB,0.012169,0.100518,0.121061
HML,0.027523,0.108816,0.252929
RMW,0.044845,0.083410,0.537649
CMA,0.033260,0.071503,0.465159
UMD,0.065513,0.154486,0.424069


## 2. Based on the factor statistics above, answer the following.

### (a) Does each factor have a positive risk premium (positive expected excess return)?
Yes, each factor has a positive risk premium.
### (b) How have the factors performed since the time of the case, (2015-present)?
THe SMB and HML had poor performance since it had negative risk premium.  CMA and UMD exhibits worse performance than they had before. Overall these factors performed worse than before (except for MKT)

In [319]:
#2b) 
df_recent = df['2015':]
ss_recent = (df_recent.mean()*12).to_frame(name = 'Mean')
ss_recent['vol'] = (df_recent.std()*np.sqrt(12))
ss_recent['Sharpe'] = ss_recent['Mean']/ss_recent['vol']

ss_recent

,Mean,vol,Sharpe
MKT,0.106943,0.160183,0.667635
SMB,-0.005765,0.097683,-0.059020
HML,-0.019709,0.132493,-0.148753
RMW,0.039535,0.071200,0.555262
CMA,0.002243,0.079622,0.028177
UMD,0.025526,0.136844,0.186534


## 3. Report the correlation matrix across the six factors.

### (a) Does the construction method succeed in keeping correlations small?
Overall the correlations are fairly insignificant, while the correlation between HML and CMA is as high as 0.67.

### (b) Fama and French say that HML is somewhat redundant in their 5-factor model. Does this seem to be the case?
I can see why they claim this since HML has a high correlation with CMA (0.67). But I don't think that HML is so redundant that it has to be removed from the model based on the fact that it has pretty low corrlations with facotrs apart from CMA.

In [52]:
#3a) corr matrix
df.corr()


,MKT,SMB,HML,RMW,CMA,UMD
MKT,1.000000,0.226318,-0.222080,-0.255413,-0.381869,-0.167707
SMB,0.226318,1.000000,-0.072132,-0.414346,-0.064181,-0.030372
HML,-0.222080,-0.072132,1.000000,0.229527,0.672451,-0.234881
RMW,-0.255413,-0.414346,0.229527,1.000000,0.115515,0.075281
CMA,-0.381869,-0.064181,0.672451,0.115515,1.000000,-0.012195
UMD,-0.167707,-0.030372,-0.234881,0.075281,-0.012195,1.000000


## 4. Report the tangency weights for a portfolio of these 6 factors.
### (a) Which factors seem most important? And Least?
It seems that CMA,RMW and MKT are most important, while the rest are less important.
### (b) Are the factors with low mean returns still useful?
Yes, we can observe that even CMA has mean as low as 0.033260, it still takes about 0.37 weights in the tangency portfolio，which is certainly important.
### (c) Re-do the tangency portfolio, but this time only include MKT, SMB, HML, and UMD. Which factors get high/low tangency weights now?
Now HML has the most siginificant weight and SMB has the least significant weight.

In [320]:
#4 a) The tangency weights for a portfolio 

#covariance vector
cov = df.cov()

#number of assets
N = cov.shape[1]

cov_inverse = np.linalg.inv(cov)
mean = df.mean()
weights = cov_inverse @ mean/((np.ones(N))@cov_inverse@mean)
omega_tangency = pd.Series(weights, index=mean.index)
omega_tangency.to_frame('Tangency weights')




,Tangency weights
MKT,0.201062
SMB,0.081551
HML,-0.047037
RMW,0.288377
CMA,0.377449
UMD,0.098597


In [321]:
#4c) Redo the tangency portfolio
df_1  = df.copy()
del df_1['RMW']
del df_1['CMA']
new_factors = df_1

cov = new_factors.cov()

#number of assets
N = cov.shape[1]

cov_inverse = np.linalg.inv(cov)
mean = new_factors.mean()
weights = cov_inverse @ mean/((np.ones(N))@cov_inverse@mean)
omega_tangency = pd.Series(weights, index=mean.index)
omega_tangency.to_frame('New tangency weights')




,New tangency weights
MKT,0.331433
SMB,0.006051
HML,0.362221
UMD,0.300295


### What do you conclude about the importance or unimportance of these styles?
The individual return of these factors could be not very important, but the correlatoion between the factors are important.

# Part 3 Testing Modern LPMs

In [153]:
portfolios = pd.read_excel('factor_pricing_data.xlsx',sheet_name = 2)
portfolios.set_index('Date',inplace  = True)
portfolios
AQR = ['MKT','HML','RMW','UMD']
CAPM = ['MKT']
Three_Factor_Model = ['MKT','SMB','HML']
five_Fama_French = ['MKT','SMB','HML','RMW','CMA']
portfolios.head()


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-31,-0.0050,0.0283,0.0084,0.1024,-0.0143,0.0999,0.0354,0.0352,0.0048,0.0032,...,0.0159,0.0876,0.0463,-0.0116,0.0458,-0.0279,0.0258,0.0751,0.0299,0.0665
1980-02-29,0.0111,-0.0610,-0.0966,-0.0319,-0.0569,-0.0314,-0.0527,-0.0788,-0.0556,-0.0140,...,-0.0079,-0.0535,-0.0339,-0.0633,-0.0638,-0.0855,-0.0960,-0.0314,-0.0275,-0.0267
1980-03-31,-0.2244,-0.1116,-0.0167,-0.1464,-0.0192,-0.1281,-0.0817,-0.1278,-0.0565,-0.0664,...,-0.0821,-0.1511,-0.1106,-0.0922,-0.1443,-0.0563,-0.0883,-0.2441,-0.1245,-0.1728
1980-04-30,0.0451,0.0766,0.0232,0.0305,0.0831,-0.0521,0.0775,0.0182,0.0304,0.0113,...,0.0419,-0.0097,-0.0300,0.0351,0.0522,0.0729,0.0532,0.0997,0.0448,0.0762
1980-05-31,0.0637,0.0792,0.0457,0.0895,0.0814,0.0512,0.0324,0.0876,0.0560,0.0064,...,0.0565,0.1060,0.1147,0.0868,0.1127,0.0577,0.0557,0.1040,0.0839,0.0684


In [322]:
#1 a)
def ts_test(df, factor_df, factors, test, annualization=12):
    res = pd.DataFrame(data = None, index = df.columns, columns = [test + r' $\alpha$', test + r' $R^{2}$'])
    
    for port in df.columns:
        y = df[port]
        X = sm.add_constant(factor_df[factors])
        model = sm.OLS(y, X).fit()
        res.loc[port] = [model.params[0] * annualization, model.rsquared] 
    return res

ts_test(portfolios, df, AQR, 'AQR')






,AQR $\alpha$,AQR $R^{2}$
Agric,0.015617,0.330166
Food,0.015205,0.468121
Soda,0.023801,0.309781
Beer,0.026757,0.424796
Smoke,0.039908,0.257474
Toys,-0.027704,0.503276
Fun,0.027091,0.615578
Books,-0.029218,0.688613
Hshld,-0.00089,0.568142
Clths,-0.001364,0.618474


In [323]:
#1b) MAE
#absolute values

res1 = res.iloc[:,0].abs().to_frame()
N = res1.shape[0]
AQR_MAE = res1.sum()/N
AQR_MAE

Market Beta    0.0
dtype: object

## 1b)  If the pricing model worked, should these alpha estimates be large or small? Why? Based on your MAE stat, does this seem to support the pricing model or not?
MAE should be small, in this case MAE is quite small so it seems to support the pricing model.

## 2. Test the CAPM, FF 3-Factor Model and the the FF 5-Factor Model. Report the MAE statistic for each of these models and compare it with the AQR Model MAE. Which model  ts best?
We can see that the CAPM has the least MAE, so it fits the best.

In [324]:
factor_tests = ts_test(portfolios, df, CAPM, 'CAPM').join(ts_test(portfolios, df, Three_Factor_Model, 'Fama-French 3F'))\
                                                         .join(ts_test(portfolios, df, five_Fama_French, 'Fama-French 5F'))
factors_MAE = factor_tests[[r'CAPM $\alpha$',
                            r'Fama-French 3F $\alpha$',
                            r'Fama-French 5F $\alpha$']].abs().mean().to_frame('MAE')
factors_MAE.index = ['CAPM','Fama-French 3F','Fama-French 5F']

factors_MAE
factor_tests



,CAPM $\alpha$,CAPM $R^{2}$,Fama-French 3F $\alpha$,Fama-French 3F $R^{2}$,Fama-French 5F $\alpha$,Fama-French 5F $R^{2}$
Agric,0.027558,0.323423,0.022531,0.346552,0.008669,0.350883
Food,0.054032,0.364266,0.045295,0.416249,0.004724,0.493345
Soda,0.051889,0.253812,0.039306,0.280149,0.001595,0.312767
Beer,0.067282,0.336213,0.064901,0.366306,0.018097,0.447034
Smoke,0.08212,0.180856,0.069518,0.222621,0.01124,0.291025
Toys,-0.027049,0.491605,-0.028883,0.518101,-0.060395,0.540097
Fun,0.004875,0.600021,0.001779,0.61647,0.013509,0.620897
Books,-0.014594,0.654171,-0.027857,0.69395,-0.046454,0.706463
Hshld,0.02505,0.507258,0.023208,0.522483,-0.015172,0.589292
Clths,0.013873,0.557288,0.006687,0.569914,-0.020975,0.627444


## 3. Does any particular factor seem especially important or unimportant for pricing? Do you think Fama and French should use the Momentum Factor?
It is obvious that market factor plays an  especially important role for pricing. 
I think Fama and French should introduce the Momentum Factor since the model fits better with it.

## 4. This does not matter for pricing, but report the average (across n estimations) of the time-series regression r-squared statistics. Do this for each of the three models you tested. Do these models lead to high time-series r-squared stats? That is, would these factors be good in a Linear Factor Decomposition of the assets?
As we can see below, These models do not lead to high time-series r-squared stats. So they would not be good in a Linear Factor Decomposition of the assets.


In [325]:
# average of time-series R^2
AVE_R = pd.DataFrame(data = None, columns = ['CAPM','Fama-French 3F','Fama-French 5F'],index = ['average R^2'])
AVE_R['CAPM'] = factor_tests.iloc[:,1].mean()
AVE_R['Fama-French 3F'] = factor_tests.iloc[:,3].mean()
AVE_R['Fama-French 5F'] = factor_tests.iloc[:,5].mean()
AVE_R

,CAPM,Fama-French 3F,Fama-French 5F
average R^2,0.527498,0.571061,0.59636


## 5. We tested three models using the time-series tests (focusing on the time-series alphas.) Re-test these models, but this time use the cross-sectional test.

In [200]:
portfolios.head(3)


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-31,-0.0050,0.0283,0.0084,0.1024,-0.0143,0.0999,0.0354,0.0352,0.0048,0.0032,...,0.0159,0.0876,0.0463,-0.0116,0.0458,-0.0279,0.0258,0.0751,0.0299,0.0665
1980-02-29,0.0111,-0.0610,-0.0966,-0.0319,-0.0569,-0.0314,-0.0527,-0.0788,-0.0556,-0.0140,...,-0.0079,-0.0535,-0.0339,-0.0633,-0.0638,-0.0855,-0.0960,-0.0314,-0.0275,-0.0267
1980-03-31,-0.2244,-0.1116,-0.0167,-0.1464,-0.0192,-0.1281,-0.0817,-0.1278,-0.0565,-0.0664,...,-0.0821,-0.1511,-0.1106,-0.0922,-0.1443,-0.0563,-0.0883,-0.2441,-0.1245,-0.1728


In [334]:
#cross-sectional test for MKT
# y is the mean return for N portfolios
# x is the beta we just get
def ts_test(df, factor_df, factors, test, annualization=12):
    res1 = pd.DataFrame(data = None, index = df.columns, columns = ["CAPM_Beta"])
    
    for port in df.columns:
        y = df[port]
        X = sm.add_constant(factor_df[factors])
        model = sm.OLS(y, X).fit()
        res1.loc[port] = [model.params['MKT'] ] 
    return res1

a = ts_test(portfolios, df, CAPM, 'CAPM')

a

,CAPM_Beta
Agric,0.785539
Food,0.580285
Soda,0.708779
Beer,0.630407
Smoke,0.607434
Toys,1.108948
Fun,1.286579
Books,1.056216
Hshld,0.692949
Clths,1.055631


In [342]:
y = portfolios.mean()

X = sm.add_constant(a['CAPM_Beta'])

